In [15]:
# Dependencies

from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
import requests
!which chromedriver
import time

/usr/local/bin/chromedriver


In [16]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News

In [5]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [6]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text
news = soup.find_all('li', class_='slide')

latest_news_title = news[0].find('div', class_='content_title').find('a').get_text()
latest_news_p =  news[0].find('div', class_='article_teaser_body').get_text() 
print(latest_news_title)
print(latest_news_p)

#In case that we need all the news
#news_items = []
#for item in news:
#    news_items.append({ "news_title" : item.find('div', class_='content_title').find('a').get_text(),
#        "news_p" : item.find('div', class_='article_teaser_body').get_text() } )

The 'Claw Game' on Mars: NASA InSight Plays to Win
NASA's InSight lander will be the first mission to use a robotic arm to grasp instruments from the spacecraft and place them on the surface of another planet. 


# JPL Mars Space Images - Featured Image

In [7]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
browser.find_link_by_partial_text("FULL IMAGE").click()
time.sleep(10)
browser.find_link_by_partial_text("more info").click()
time.sleep(5)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [8]:
url = soup.find('img', class_="main_image")['src']
featured_image_url = "https://www.jpl.nasa.gov" + url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA09178_hires.jpg'

# Mars Weather

In [17]:
# URL of page to be scraped
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [18]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [19]:
tweets = soup.find_all('div', class_='js-tweet-text-container')
#tweets = soup.find_all('p', class_='tweet-text')
mars_weather = ""

for tweet in tweets:
    if tweet.p.get_text().startswith("Sol "):
        #print(tweet)
        mars_weather = tweet.p.get_text()
        break
    
mars_weather

'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59'

# Mars Facts

In [11]:
url = "https://space-facts.com/mars"
browser.visit(url)
# HTML object
html = browser.html

In [12]:
mars_facts_df = pd.read_html(html)[0]
mars_facts_df.columns = ["Description", "Value"]
mars_facts_df.set_index("Description", inplace=True)
mars_facts_html = mars_facts_df.to_html(na_rep = " ", classes="table table-sm table-striped", justify="left", col_space=0)
mars_facts_html

'<table border="1" class="dataframe table table-sm table-striped">\n  <thead>\n    <tr style="text-align: left;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian

# Mars Hemispheres

In [13]:
hemisphere_image_urls = []
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_hemispheres = []
[mars_hemispheres.append("https://astrogeology.usgs.gov" + item_url.a['href']) for item_url in soup.findAll("div", class_="description")]
mars_hemispheres

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [14]:
for url in mars_hemispheres:
    browser.visit(url)
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    images = soup.find('div', class_='wide-image-wrapper').find_all('li')
    img_url = images[0].a['href']
    hemisphere_image_urls.append( { "title" : soup.find('h2', class_='title').text, "img_url" : img_url })

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]